In [12]:
import tensorflow as tf
import tensorflow.keras as tfk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from WeightedBinaryCrossentropy import WeightedBinaryCrossentropy

In [13]:
model = tfk.Sequential()
model.add(tfk.Input(shape = (4,4)))
model.add(tfk.layers.Bidirectional(tfk.layers.LSTM(128, return_sequences=True)))
model.add(tfk.layers.ReLU())
model.add(tfk.layers.Dropout(0.3))
model.add(tfk.layers.Bidirectional(tfk.layers.LSTM(256, return_sequences=True)))
model.add(tfk.layers.ReLU())
model.add(tfk.layers.Dropout(0.3))
model.add(tfk.layers.Bidirectional(tfk.layers.LSTM(256, return_sequences=True)))
model.add(tfk.layers.ReLU())
model.add(tfk.layers.Dropout(0.3))
model.add(tfk.layers.LSTM(128, return_sequences=False))
model.add(tfk.layers.ReLU())
model.add(tfk.layers.Dense(1))

In [14]:
model.load_weights(r'..\Pretrained Weights\po_hour')

In [15]:
data = pd.read_csv(r'..\DataRetrieval\weather_data_new.csv')
data.drop(columns=['location'], inplace=True)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      5 non-null      datetime64[ns]
 1   temperature    5 non-null      float64       
 2   humidity       5 non-null      int64         
 3   wind_speed     5 non-null      float64       
 4   precipitation  5 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 328.0 bytes


In [16]:
from fun import prepro
X = prepro(data)

In [17]:
gen = tfk.preprocessing.sequence.TimeseriesGenerator(np.asarray(X),np.asarray(X),length=4,sampling_rate=1,stride=1,batch_size=32)

In [18]:
yhat = model.predict(gen)

1/1 [==============================] - 3s 3s/step


In [19]:
yhat = [1 if x > 0.5 else 0 for x in yhat]
yhat

[0]

In [20]:
if yhat[0] == 0:
    print('NO DETECTABLE POWER OUTAGE IN THE COMING HOUR')
else:
    print('POSSIBILITY OF POWER OUTAGE IN THE COMING HOUR')

NO DETECTABLE POWER OUTAGE IN THE COMING HOUR
